In [ ]:
!pip install --upgrade pip
!pip install gensim
!pip install nltk
!pip install tokenizers
!pip install sentencepiece
!pip install python-bidi
!pip install arabic-reshaper
!pip install PyArabic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 3.1 MB/s eta 0:00:00


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import pandas as pd
# from sklearn.metrics import accuracy_score
import numpy as np
from utilities import *
import textProcessing as tp
import nltk
# nltk.download('punkt')
train_text = load_text("dataset/train.txt")
tp.preprocessing_text(train_text,"train_preprocessed.txt")


['قَوْلُهُ :  أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ  قَالَ الزَّرْكَشِيُّ    ',
 'ابْنُ عَرَفَةَ : قَوْلُهُ : بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً  كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ  ابْنُ عَرَفَةَ : قَوْلُ ابْنِ شَاسٍ : أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ  وَسِحْرٍ  مُحَمَّدٌ : قَوْلُ مَالِكٍ وَأَصْحَابِهِ أَنَّ السَّاحِرَ كَافِرٌ بِاَللَّهِ تَعَالَى قَالَ مَالِكٌ : هُوَ كَالزِّنْدِيقِ إذَا عَمِلَ السِّحْرَ بِنَفْسِهِ قُتِلَ وَلَمْ يُسْتَتَبْ .',
 ' قَوْلُهُ لِعَدَمِ مَا تَتَعَلَّقُ إلَخْ  أَيْ الْوَصِيَّةُ  قَوْلُهُ مَا مَرَّ  أَيْ قُبَيْلَ قَوْلِ الْمَتْنِ لَغَتْ وَلَوْ اقْتَصَرَ عَلَى أَوْصَيْت لَهُ بِشَاةٍ أَوْ أَعْطُوهُ شَاةً وَلَا غَنَمَ لَهُ عِنْدَ الْمَوْتِ هَلْ تَبْطُلُ الْوَصِيَّةُ أَوْ يُشْتَرَى لَهُ شَاةٌ وَيُؤْخَذُ مِنْ قَوْلِهِ الْآتِي كَمَا لَوْ لَمْ يَقُلْ مِنْ مَالِي وَلَا مِ

In [2]:
class MyDataset(Dataset):
    def __init__(self):
        self.data = load_text("dataset/train_preprocessed.txt")
        self.transform = ToTensor()
        self.T = 50

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        sentence = self.data[idx]
        # extract the label
        labels,sentence= tp.extract_diacritics_with_previous_letter(sentence)

        # remove the label from the sentence
        # sentence = tp.clear_diacritics(sentence)

        # convert the sentence to one hot encoding
        sentence = convert_sentence_to_vector(sentence)

        # convert the labels to one hot encoding
        labels = convert_labels_to_vector(labels)

        # pad the sentence and labels if smaller than T
        temp = np.zeros((self.T, 38))
        if self.T > sentence.shape[0]:
            temp[:sentence.shape[0], :] = sentence
            # append the sentence with spaces
            temp[sentence.shape[0]:, 36] = 1
            sentence = temp
        else:
            sentence = sentence[:self.T, :]
        temp = np.zeros((self.T, 15))
        if self.T > labels.shape[0]:
            temp[:labels.shape[0], :] = labels
            labels = temp
        else:
            labels = labels[:self.T, :]
            # append the labels with no diacritics
            labels[self.T:, 14] = 1 

        # convert the sentence and labels to tensors
        sentence = torch.tensor(sentence, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.float32)

        return sentence, labels


In [3]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNClassifier, self).__init__()

        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_layers = 5
        self.rnn = nn.RNN(input_size, hidden_size,num_layers=self.num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 15)
        self.init_weight()

    def forward(self, input, hidden):
        output, hidden = self.rnn(input, hidden)
        output = self.linear(output)
        return output

    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers,batch_size, self.hidden_size)
    def init_weight(self):
        for name, param in self.named_parameters():
            if 'bias' in name:
                nn.init.constant(param, 0.0)
            elif 'weight' in name:
                nn.init.xavier_normal(param)

In [5]:
class LSTMClassifier(nn.Module):
    def _init_(self, input_size, hidden_size, output_size):
        super(LSTMClassifier, self)._init_()

        self.hidden_size = hidden_size
        self.input_size = input_size

        self.lstm = nn.LSTM(input_size, hidden_size, 5,batch_first=True)
        self.tanh = nn.Tanh()
        self.linear = nn.Linear(hidden_size, output_size)
        self.init_weight()

    def forward(self, input, hidden, c):
        output, _ = self.lstm(input, (hidden,c))
        # reshape the output to be able to pass it to the linear layer
        # output = output.contiguous().view(-1, self.hidden_size)
        output = self.tanh(output)
        output = self.linear(output)
        # output = self.soft(output)
        return output

    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size)
    def init_c(self, batch_size):
        return torch.zeros(1,batch_size, self.hidden_size)
    def init_weight(self):
        for name, param in self.named_parameters():
            if 'bias' in name:
                nn.init.constant(param, 0.0)
            elif 'weight' in name:
                nn.init.xavier_normal(param)

In [4]:
# Hyperparameter
input_size = 38
hidden_size = 128
output_size = 15
batch_size = 1024
num_epochs = 500

In [6]:
# connect to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.is_available())
print(torch.cuda.device_count())

# Create an instance of the RNN classifier
model = RNNClassifier(input_size, hidden_size, output_size)
model.to(device)

# Load data from CSV file
dataset = MyDataset()

# Create a dataloader to handle batching and shuffling
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=1, gamma=0.5)

cuda:0
True
1


C:\Users\ahmed sabry\AppData\Local\Temp\ipykernel_15592\1453550816.py:24: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(param)
C:\Users\ahmed sabry\AppData\Local\Temp\ipykernel_15592\1453550816.py:22: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(param, 0.0)


In [7]:
# Training loop
# optimizer = optim.Adam(model.parameters(), lr=0.00005)
# model.load_state_dict(torch.load('model1.pth'))
model.train()
for epoch in range(num_epochs):
    scheduler.step()
    for inputs, labels in train_dataloader:
      # Reshape input and labels to (batch_size, seq_length, input_size)
      input = inputs.view(inputs.shape[0], -1, input_size)
      labels = labels.view(inputs.shape[0], -1, output_size)
      input, labels = input.to(device), labels.to(device)
      hidden = model.init_hidden(batch_size=inputs.shape[0])
      hidden = hidden.to(device)

      # Forward pass
      output = model(input, hidden)

      # Compute loss
      loss = criterion(output, labels)

      # Zero the gradients
      optimizer.zero_grad()

      # Backward pass and optimization
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
      optimizer.step()
      


      # Print loss for monitoring
      print(f"Epoch: {epoch+1}, Batch Loss: {loss.item()}")
      
      



d:\anaconda3\envs\pt\Lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch: 1, Batch Loss: 13.172357559204102
Epoch: 1, Batch Loss: 12.504874229431152
Epoch: 1, Batch Loss: 12.401000022888184
Epoch: 1, Batch Loss: 12.189319610595703
Epoch: 1, Batch Loss: 12.096535682678223
Epoch: 1, Batch Loss: 12.058197975158691
Epoch: 1, Batch Loss: 11.927419662475586
Epoch: 1, Batch Loss: 12.013023376464844
Epoch: 1, Batch Loss: 11.751697540283203
Epoch: 1, Batch Loss: 11.706832885742188
Epoch: 1, Batch Loss: 11.640981674194336
Epoch: 1, Batch Loss: 11.536144256591797
Epoch: 1, Batch Loss: 11.457464218139648
Epoch: 1, Batch Loss: 11.235041618347168
Epoch: 1, Batch Loss: 11.327306747436523
Epoch: 1, Batch Loss: 11.168658256530762
Epoch: 1, Batch Loss: 11.000543594360352
Epoch: 1, Batch Loss: 11.004928588867188
Epoch: 1, Batch Loss: 10.96420955657959
Epoch: 1, Batch Loss: 11.07912826538086
Epoch: 1, Batch Loss: 10.930150032043457
Epoch: 1, Batch Loss: 10.665763854980469
Epoch: 1, Batch Loss: 10.795795440673828
Epoch: 1, Batch Loss: 10.774237632751465
Epoch: 1, Batch Lo

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'model1.pth')

In [8]:
# Set the model to evaluation mode
# model.load_state_dict(torch.load('model1.pth'))
model.eval()

# Iterate over the test dataset
predictions = []
ground_truth = []
correct = 0
correct_prediction = 0
total_predictions = 0
total = 0

with torch.no_grad():
  for inputs, labels in train_dataloader:
      # Forward pass
      hidden = model.init_hidden(batch_size=inputs.shape[0])
      hidden = hidden.to(device)
      inputs ,labels = inputs.to(device),labels.to(device)
      outputs = model(inputs,hidden)
      outputs = outputs.view(inputs.shape[0], -1, output_size)
      # Get predictions from the maximum value
      _, predicted = torch.max(outputs.data, 2)
      predicted = predicted.cpu()
      labels = labels.cpu()
      # Convert to numpy arrays
      predictions.extend(predicted.numpy())

      # get labels from one hot encoding
      _, labels = torch.max(labels.data, 2)
      # Convert to numpy arrays
      ground_truth.extend(labels.numpy())
      # Compute accuracy
      correct += (predicted == labels).sum().item()
      total += labels.size(0) * labels.size(1)
      print(f"Accuracy: {correct/total}")
      # break
      

print(f"Accuracy: {correct/total}")

# Convert lists to numpy arrays
predictions = np.array(predictions)
ground_truth = np.array(ground_truth)

Accuracy: 0.566953125
Accuracy: 0.561201171875
Accuracy: 0.55955078125
Accuracy: 0.558037109375
Accuracy: 0.5573125
Accuracy: 0.5577213541666667
Accuracy: 0.5564871651785714
Accuracy: 0.55744140625
Accuracy: 0.5564539930555555
Accuracy: 0.55639453125
Accuracy: 0.5561612215909091
Accuracy: 0.5556998697916666
Accuracy: 0.555381610576923
Accuracy: 0.5548563058035715


KeyboardInterrupt: 

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(ground_truth, predictions)
print(f"Accuracy: {accuracy}")

In [116]:
# clear the memory of the GPU
import gc
torch.cuda.empty_cache()
gc.collect()

1057